# HackDavis python library tests

## OCS-HackDavis Package Installation 

In [7]:
!pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ ocs-hackdavis==0.18.0

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple/


## Import functions

In [8]:
from ocs_hackdavis import (
    ucdavis_buildings,
    ucdavis_ceeds_of,
    ucdavis_dataview_id,
    ucdavis_config_data,
)

## List of buildings

In [9]:
buildings = ucdavis_buildings()
len(buildings), buildings[:25]

(162,
 ['ARC Pavilion',
  'Academic Surge Building',
  'Activities and Recreation Center',
  'Advanced Materials Research Laboratory',
  'Advanced Transportation Infrastructure Research Center',
  'Aggie Stadium',
  'Agronomy Field Laboratory',
  'Animal Building',
  'Animal Resource Service J1',
  'Animal Resource Service M3',
  'Animal Resource Service N1',
  'Ann E. Pitzer Center',
  'Antique Mechanics Trailer',
  'Aquatic Biology & Environmental Science Bldg',
  'Art Building',
  'Art Building Annex',
  'Art, Music, Wright Halls',
  'Asmundson Annex',
  'Asmundson Hall',
  'Bainer Hall',
  'Bowley Head House',
  'Briggs Hall',
  'California Hall',
  'Campus Data Center',
  'Cellular Biology Laboratory'])

## Function `ucdavis_ceeds_of` returns list of CEEDs for a given building

### Display them for all buildings

In [10]:
for num, building in enumerate(ucdavis_buildings(), 1):
    print(f"[{num}] Building: {building} ==> CEEDS: {ucdavis_ceeds_of(building)}")

[1] Building: ARC Pavilion ==> CEEDS: ['Baseline Electricity', 'ChilledWater', 'Electricity', 'Steam']
[2] Building: Academic Surge Building ==> CEEDS: ['ChilledWater', 'Electricity', 'Steam']
[3] Building: Activities and Recreation Center ==> CEEDS: ['Baseline ChilledWater', 'Baseline Electricity', 'Baseline Steam', 'ChilledWater', 'Electricity', 'Steam']
[4] Building: Advanced Materials Research Laboratory ==> CEEDS: ['Baseline Electricity', 'Electricity']
[5] Building: Advanced Transportation Infrastructure Research Center ==> CEEDS: ['Baseline Electricity', 'Electricity']
[6] Building: Aggie Stadium ==> CEEDS: ['Electricity']
[7] Building: Agronomy Field Laboratory ==> CEEDS: ['Electricity']
[8] Building: Animal Building ==> CEEDS: ['Electricity']
[9] Building: Animal Resource Service J1 ==> CEEDS: ['Electricity', 'Natural Gas']
[10] Building: Animal Resource Service M3 ==> CEEDS: ['Electricity']
[11] Building: Animal Resource Service N1 ==> CEEDS: ['Electricity']
[12] Building: An

## Test error code 

### Bad argument of the right type

In [11]:
# should fail
ucdavis_ceeds_of("Bad Building")

#warning: "Bad Building" is not a valid building name


## Bad argument type

In [12]:
# should fail - type error
ucdavis_ceeds_of(42)

TypeError: type of argument "building" must be str; got int instead

## Get Data View ID of a given building+CEED pair

In [13]:
dv_id = ucdavis_dataview_id("Activities and Recreation Center", "Electricity")
dv_id

'hackdavis_9747d9db'

## Import for OCS Sample Library and other necessary modules

In [14]:
from ocs_sample_library_preview import OCSClient
import configparser
import io
import json
import pandas as pd

## Standard configuration file parsing and OCS client object 

#### Reference: https://github.com/osisoft/OSI-Samples-OCS/blob/master/basic_samples/DataViews/Python3/program.py#L150

## IMPORTANT: REPLACE CLIENT ID + SECRET BELOW BEFORE RUNNING

In [25]:
config_text = u"""
; IMPORTANT: replace these values with those provided by OSIsoft
[Configurations]
Namespace = UC__Davis

[Access]
Resource = https://dat-b.osisoft.com
Tenant = 65292b6c-ec16-414a-b583-ce7ae04046d4
ApiVersion = v1-preview

[Credentials]
ClientId = REPLACE_ME
ClientSecret = REPLACE_ME
"""

In [26]:
config = configparser.ConfigParser(allow_no_value=True)
config.read_file(io.StringIO(config_text))

ocs_client = OCSClient(
    config.get("Access", "ApiVersion"),
    config.get("Access", "Tenant"),
    config.get("Access", "Resource"),
    config.get("Credentials", "ClientId"),
    config.get("Credentials", "ClientSecret"),
)

namespace_id = config.get("Configurations", "Namespace")
print(f"namespace_id: '{namespace_id}'")

SdsError: 'Failed to get token, check client id/secret: invalid_client'

## Getting CSV data from Data View

In [19]:
# Step 1) get data view id
dv_id = ucdavis_dataview_id("Activities and Recreation Center", "Electricity")

# Step 2) call data view
csv, token = ocs_client.DataViews.getDataInterpolated(
    namespace_id,
    dv_id,
    startIndex="2018-01-01",
    endIndex="2018-05-01",
    interval="00:20:00",
    count=250000,
    form="csvh",
)
# make sure there is no additional data
assert token is None

# display result subset
print(len(csv), csv[:2048])

1307501 Timestamp,Building,CEED,Demand_kBtu,Electricity_EUI,MonthlyUsage,Cumulative Use,Demand,AnnualUsage,Rollover Check,Rollover Count Year,Rollover Count Month,AnnualCost,EUI
2018-01-01T00:00:00.0000000Z,Activities and Recreation Center,Electricity,482.0144,42.8698,484380.3,5538609.89475807,141.212522525084,6773462,0,0,0,,
2018-01-01T00:20:00.0000000Z,Activities and Recreation Center,Electricity,484.548,42.86924,484533.3,5538657.99972426,142.012899645591,6773400,0,0,0,,
2018-01-01T00:40:00.0000000Z,Activities and Recreation Center,Electricity,514.763,42.86868,484686.3,5538706.10469045,150.868408203125,6773337,0,0,0,,
2018-01-01T01:00:00.0000000Z,Activities and Recreation Center,Electricity,517.5825,42.86811,484839.3,5538755.10315018,151.694748312057,6773275,0,0,0,,
2018-01-01T01:20:00.0000000Z,Activities and Recreation Center,Electricity,524.0535,42.86755,484992.4,5538806.13010686,153.622489675805,6773213,0,0,0,,
2018-01-01T01:40:00.0000000Z,Activities and Recreation Center,Electric

## Getting a Pandas dataframe from CSV

In [20]:
# Step 3) transform CSV into a Pandas dataframe, convert Timestamp column to the right data type
df = pd.read_csv(io.StringIO(csv), parse_dates=['Timestamp'])
df

,Timestamp,Building,CEED,Demand_kBtu,Electricity_EUI,MonthlyUsage,Cumulative Use,Demand,AnnualUsage,Rollover Check,Rollover Count Year,Rollover Count Month,AnnualCost,EUI
0,2018-01-01 00:00:00+00:00,Activities and Recreation Center,Electricity,482.0144,42.86980,484380.3,5.538610e+06,141.212523,6773462,0,0,0.0,NaN,NaN
1,2018-01-01 00:20:00+00:00,Activities and Recreation Center,Electricity,484.5480,42.86924,484533.3,5.538658e+06,142.012900,6773400,0,0,0.0,NaN,NaN
2,2018-01-01 00:40:00+00:00,Activities and Recreation Center,Electricity,514.7630,42.86868,484686.3,5.538706e+06,150.868408,6773337,0,0,0.0,NaN,NaN
3,2018-01-01 01:00:00+00:00,Activities and Recreation Center,Electricity,517.5825,42.86811,484839.3,5.538755e+06,151.694748,6773275,0,0,0.0,NaN,NaN
4,2018-01-01 01:20:00+00:00,Activities and Recreation Center,Electricity,524.0535,42.86755,484992.4,5.538806e+06,153.622490,6773213,0,0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8636,2018-04-30 22:40:00+00:00,Activities and Recreation Center,Electricity,879.9150,41.69452,552921.9,6.159769e+06,262.015037,6591898,0,0,0.0,NaN,NaN
8637,2018-04-30 23:00:00+00:00,Activities and Recreation Center,Electricity,879.9156,41.69426,553185.9,6.159858e+06,258.040084,6591902,0,0,0.0,NaN,NaN
8638,2018-04-30 23:20:00+00:00,Activities and Recreation Center,Electricity,879.9163,41.69400,553449.8,6.159947e+06,271.704426,6591906,0,0,0.0,NaN,NaN
8639,2018-04-30 23:40:00+00:00,Activities and Recreation Center,Electricity,879.9169,41.69374,553713.8,6.160035e+06,267.195156,6591910,0,0,0.0,NaN,NaN


In [21]:
# Same as above but keep only a subset of the columns
df_select = pd.read_csv(
    io.StringIO(csv),
    parse_dates=["Timestamp"],
    usecols=[
        "Timestamp",
        "AnnualUsage",
        "Cumulative Use",
        "Demand_kBtu",
        "Electricity_EUI",
        "MonthlyUsage",
    ],
)
df_select

,Timestamp,Demand_kBtu,Electricity_EUI,MonthlyUsage,Cumulative Use,AnnualUsage
0,2018-01-01 00:00:00+00:00,482.0144,42.86980,484380.3,5.538610e+06,6773462
1,2018-01-01 00:20:00+00:00,484.5480,42.86924,484533.3,5.538658e+06,6773400
2,2018-01-01 00:40:00+00:00,514.7630,42.86868,484686.3,5.538706e+06,6773337
3,2018-01-01 01:00:00+00:00,517.5825,42.86811,484839.3,5.538755e+06,6773275
4,2018-01-01 01:20:00+00:00,524.0535,42.86755,484992.4,5.538806e+06,6773213
...,...,...,...,...,...,...
8636,2018-04-30 22:40:00+00:00,879.9150,41.69452,552921.9,6.159769e+06,6591898
8637,2018-04-30 23:00:00+00:00,879.9156,41.69426,553185.9,6.159858e+06,6591902
8638,2018-04-30 23:20:00+00:00,879.9163,41.69400,553449.8,6.159947e+06,6591906
8639,2018-04-30 23:40:00+00:00,879.9169,41.69374,553713.8,6.160035e+06,6591910


## Access AF configuration data for a building+CEED pair

### Example

In [22]:
ucdavis_config_data(ocs_client, namespace_id, "Activities and Recreation Center", "Electricity")

{'Annual Cost': 151156.0,
 'BuildingName': 'ARC',
 'kWh Rate': 0.0687,
 'Prefix': 'Activities_and_Recreation_Center_MSB',
 'Rollover': 10000000.0,
 'CAAN': 4799.0,
 'Construction Date': '04/15/2002',
 'Display Name': 'Activities and Recreation Center',
 'Latitude': 38.5428969596,
 'Longitude': -121.759644393,
 'Maintained Gross Sq. Ft.': 158120.0,
 'Primary Usage (Type)': 'REC - Athletics & Recreation'}

#### Note: each CEED of a building has a copy of the AF configuration data of the parent element (e.g. BuildingName, Latitude, Longitude, etc)

In [23]:
ucdavis_config_data(ocs_client, namespace_id, "Activities and Recreation Center", "Steam")

{'Annual Cost': 46360.0,
 'BuildingName': 'ARC',
 'klb Rate': 7.2552,
 'Prefix': 'Activities_and_Recreation_Center_MSB',
 'Rollover': 100000.0,
 'Rollover Count Month': 0.0,
 'Rollover Count Year': 0.0,
 'CAAN': 4799.0,
 'Construction Date': '04/15/2002',
 'Display Name': 'Activities and Recreation Center',
 'Latitude': 38.5428969596,
 'Longitude': -121.759644393,
 'Maintained Gross Sq. Ft.': 158120.0,
 'Primary Usage (Type)': 'REC - Athletics & Recreation'}

## Display AF configuration data of 10 first building+CEEDs

In [24]:
count = 1
for building in ucdavis_buildings():
    for ceed in ucdavis_ceeds_of(building):
        print(
            f"[{count}] {building} | {ceed}\n  {ucdavis_config_data(ocs_client, namespace_id, building, ceed)}"
        )
        count += 1
        print("------------------------")
    if count >= 10:
        break

[1] ARC Pavilion | Baseline Electricity
  {'Building Name': 'ARCPavilion', 'CDD Coefficient': -0.48, 'CDD^2 Coefficient': 0.03, 'HDD Coefficient': 1.34, 'HDD^2 Coefficient': -0.06, 'Intercept Coefficient': 279.71, 'MONTH 1 Coefficient': 32.25, 'MONTH 10 Coefficient': -8.3, 'MONTH 11 Coefficient': -1.69, 'MONTH 2 Coefficient': 24.78, 'MONTH 3 Coefficient': -25.79, 'MONTH 4 Coefficient': -26.2, 'MONTH 5 Coefficient': -42.44, 'MONTH 6 Coefficient': -4.45, 'MONTH 7 Coefficient': -39.21, 'MONTH 8 Coefficient': 11.07, 'MONTH 9 Coefficient': -23.57, 'Outlier High': '', 'Outlier Low': '', 'Predicted EUI': '', 'Predicted EUI Outlier High': '', 'Predicted EUI Outlier Low': '', 'TOD 0 Coefficient': -33.44, 'TOD 1 Coefficient': -33.42, 'TOD 10 Coefficient': 43.45, 'TOD 11 Coefficient': 44.09, 'TOD 12 Coefficient': 43.51, 'TOD 13 Coefficient': 41.74, 'TOD 14 Coefficient': 40.41, 'TOD 15 Coefficient': 38.26, 'TOD 16 Coefficient': 36.35, 'TOD 17 Coefficient': 36.12, 'TOD 18 Coefficient': 37.68, 'TOD 